# Homework 4 - Part 2
> Specificity-Optimized Stimulation

In [1]:
!pip install -r requirements.txt
!nrnivmodl mechanisms

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
/usr/bin/xcrun
/Users/korashughes/Documents/GitHub/DataScience-Practice/DataDrivenAI-NeuralData/Homeworks/HW4/simulation
Mod files: "mechanisms/mechanisms/Ca.mod" "mechanisms/mechanisms/CaDynamics_E2.mod" "mechanisms/mechanisms/Ca_HVA.mod" "mechanisms/mechanisms/Ca_LVAst.mod" "mechanisms/mechanisms/Ih.mod" "mechanisms/mechanisms/Im.mod" "mechanisms/mechanisms/K_Pst.mod" "mechanisms/mechanisms/K_Tst.mod" "mechanisms/mechanisms/NaTa_t.mod" "mechanisms/mechanisms/NaTs2_t.mod" "mechanisms/mechanisms/Nap_Et2.mod" "mechanisms/mechanisms/ProbAMPANMDA_EMS.mod" "mechanisms/mechanisms/ProbGABAAB_EMS.mod" "mechanisms/mechanisms/SK_E2.mod" "mechanisms/mechanisms/SKv3_1.mod" "mechanisms/mechanisms/vecevent.mod" "mechanisms/mechanisms/xtra.mod"

 -> C

In [ ]:
from simulation_Pyr import simulation_Pyr
from simulation_PV import simulation_PV
from helper import firing_rate
import ray

import numpy as np
import pandas as pd
import plotly.express as px
from ax.service.managed_loop import optimize
from IPython.display import clear_output

## Bayes Opt

In [ ]:
# main function
total_time = 500  # total simulation time in ms
verbose=True
stored_results = []
def run_sim(params):
    # set params
    amp1 = params["a1"]
    amp2 = params["a2"]
    freq1 = params["f1"]
    freq2 = params["f2"]
    # simulate
    ray.init(ignore_reinit_error=True, num_cpus=2)  # Initialize Ray with multiple workers
    results = [simulation_Pyr.remote(
                num_electrode = 1,  
                amp1 = amp1, amp2 = amp2, freq1 = freq1, freq2 = freq2,
                total_time = total_time,  
                plot_waveform = False # Set to True to plot injected current
            ),
            simulation_PV.remote(
                num_electrode=1,  
                amp1 = amp1, amp2 = amp2, freq1 = freq1, freq2 = freq2,
                total_time = total_time,  
                plot_waveform = False # Set to True to plot injected current
            )]
    # Retrieve results from Ray
    (response_Pyr, t), (response_PV, t) = ray.get(results)
    # Compute firing rates
    FR_Pyr = firing_rate(np.array(response_Pyr), total_time)
    FR_PV = firing_rate(np.array(response_PV), total_time)
    result = FR_PV - FR_Pyr
    stored_results.append([len(stored_results)+1, amp1, amp2, freq1, freq2, result, FR_PV, FR_Pyr])
    ray.shutdown()
    
    if verbose:
        clear_output()
        print(f"*{len(stored_results)} Trials Completed:")
        for i, a, b, c, d, res, pv, pyr in stored_results:
            print(f"{[a,b,c,d]} => {res} with {pv=}, {pyr=}")
    return result

In [56]:
# main BO loop
best_parameters, values, experiment, model = optimize(
    parameters=[
        {
            "name": "a1",
            "type": "range",
            "bounds": [0.0, 200.0],
        },
        {
            "name": "a2",
            "type": "range",
            "bounds": [0.0, 200.0],
        },
        {
            "name": "f1",
            "type": "range",
            "bounds": [0.0, 100.0],
        },
        {
            "name": "f2",
            "type": "range",
            "bounds": [0.0, 100.0],
        },
    ],
    evaluation_function=run_sim,
    total_trials=20
    )

*22 Trials Completed:
[93.20727586746216, 103.26287746429443, 60.42485237121582, 84.91347432136536] => 44.0 with pv=50.0, pyr=6.0
[130.70854879915714, 143.29299721866846, 89.69263397157192, 87.39130487665534] => 52.0 with pv=72.0, pyr=20.0
[156.79566860198975, 160.49445867538452, 96.6912567615509, 0.17335349693894386] => 48.0 with pv=64.0, pyr=16.0
[99.79959726333618, 68.05569846183062, 34.632277861237526, 50.51732370629907] => 36.0 with pv=42.0, pyr=6.0
[2.477896399796009, 14.823788776993752, 39.73452541977167, 9.060632716864347] => 0.0 with pv=0.0, pyr=0.0
[47.970582731068134, 75.78974384814501, 21.373808570206165, 27.711867913603783] => 20.0 with pv=24.0, pyr=4.0
[25.537831522524357, 150.39502140134573, 16.131962463259697, 32.741070445626974] => 44.0 with pv=50.0, pyr=6.0
[20.547817274928093, 88.3846614509821, 73.7220786511898, 29.16983589529991] => 30.0 with pv=30.0, pyr=0.0
[178.03210988640785, 142.5085734575987, 81.00500628352165, 95.62899069860578] => 40.0 with pv=70.0, pyr=30.0

/Users/korashughes/Library/Python/3.9/lib/python/site-packages/ax/utils/stats/model_fit_stats.py:133: RuntimeWarning:

divide by zero encountered in divide

/Users/korashughes/Library/Python/3.9/lib/python/site-packages/ax/utils/stats/model_fit_stats.py:163: RuntimeWarning:

divide by zero encountered in scalar divide



## Graphing

In [57]:
df = pd.DataFrame(stored_results, columns=["Iter", "A1", "A2", "f1", "f2", "Objective", "PV", "Pyr"])
# calculate & add power
power = []
for i, row in df.iterrows():
    power.append(0.5*(row["A1"]**2 + row["A2"]**2))
df["Power"] = power
df

,Iter,A1,A2,f1,f2,Objective,PV,Pyr,Power
0,1,93.207276,103.262877,60.424852,84.913474,44.0,50.0,6.0,9675.409068
1,2,130.708549,143.292997,89.692634,87.391305,52.0,72.0,20.0,18808.803891
2,3,156.795669,160.494459,96.691257,0.173353,48.0,64.0,16.0,25171.676479
3,4,99.799597,68.055698,34.632278,50.517324,36.0,42.0,6.0,7295.768854
4,5,2.477896,14.823789,39.734525,9.060633,0.0,0.0,0.0,112.942342
5,6,47.970583,75.789744,21.373809,27.711868,20.0,24.0,4.0,4022.631040
6,7,25.537832,150.395021,16.131962,32.741070,44.0,50.0,6.0,11635.421651
7,8,20.547817,88.384661,73.722079,29.169836,30.0,30.0,0.0,4117.030587
8,9,178.032110,142.508573,81.005006,95.628991,40.0,70.0,30.0,26002.062830
9,10,169.148645,103.072248,41.271762,65.134361,42.0,68.0,26.0,19617.576201


In [58]:
px.line(df, x="Iter", y="Objective", title="Objective Improvement per Iteration")

In [59]:
px.line(df, x="Iter", y=["PV", "Pyr"], title="Neuron Firing Rates per Iteration")

In [63]:
for param in ["A1", "A2", "f1", "f2"]:
    plt = px.line(df, x="Iter", y=param, title=f"Parameter {param} Choice per Iteration")
    plt.show()

In [61]:
# graph power
px.line(df, x="Iter", y="Power", title="Power of Injected Current per Iteration")